构造一个没有任何超参数的自定义层

In [1]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

将层作为组件合并到构建复杂的模型中

In [2]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean()

tensor(5.0059e-09, grad_fn=<MeanBackward0>)

带参数的图层

In [3]:
class MyLinear(nn.Module):
    def __init__(self,in_units,units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units,units))
        #print(type(units)) # <class 'int'>
        #print(type((units,))) # <class 'tuple'>
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self,X):
        #在forward中使用.data会断开梯度连接
        linear = torch.matmul(X,self.weight.data) + self.bias.data
        return F.relu(linear)

dense = MyLinear(5,3)
dense.weight

Parameter containing:
tensor([[ 1.0076, -0.5744,  1.1149],
        [ 0.5604, -0.8797, -0.5302],
        [ 0.3781,  0.2633, -0.4773],
        [ 0.5055, -1.0165, -1.2464],
        [ 0.3799, -0.3798, -0.2226]], requires_grad=True)

使用自定义层直接执行正向传播计算

In [4]:
dense(torch.rand(2,5))

tensor([[2.3752, 0.0000, 0.0000],
        [2.6127, 0.0000, 0.0000]])

使用自定义层构建模型

In [5]:
net = nn.Sequential(MyLinear(64,8),MyLinear(8,1))
net(torch.rand(2,64))

tensor([[11.7555],
        [ 5.5910]])